<a href="https://colab.research.google.com/github/dead10GOD/ollama_test/blob/main/flaskspringboottested1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh # download ollama api
from IPython.display import clear_output

# Create a Python script to start the Ollama API server in a separate thread

import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
pciutils is already the newest version (1:3.7.0-6).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [26]:
from IPython.display import clear_output
!ollama pull llama3.1:8b
clear_output()


In [27]:
!pip install -U lightrag[ollama]

from lightrag.core.generator import Generator
from lightrag.core.component import Component
from lightrag.core.model_client import ModelClient
from lightrag.components.model_client import OllamaClient, GroqAPIClient

import time


qa_template = r"""
You are a helpful assistant.

User: {{input_str}}
You:"""

class SimpleQA(Component):
    def __init__(self, model_client: ModelClient, model_kwargs: dict):
        super().__init__()
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=qa_template,
        )

    def call(self, input: dict) -> str:
        return self.generator.call({"input_str": str(input)})

    async def acall(self, input: dict) -> str:
        return await self.generator.acall({"input_str": str(input)})

In [28]:
from lightrag.components.model_client import OllamaClient
from IPython.display import Markdown, display
model = {
    "model_client": OllamaClient(),
    "model_kwargs": {"model": "llama3.1:8b"}
}
# Assuming there's code before this snippet that defines 'model' and 'SimpleQA'
# and that the '}' is erroneous.

qa = SimpleQA(**model)
output=qa("""Consider the case **Ramana Dayaram Shetty vs The International Airport Authority of India** decided on May 4, 1979. Your task is to reconstruct the case proceedings in a detailed, step-by-step narrative format.
For each significant part of the case:")
1. Present the case proceedings in a clear and engaging way.
2. Ask an analytical or opinion-based question related to the preceding part. These questions should be concise and fall into one of the following categories:
   - Agree/Disagree type
   - Analytical opinion type (e.g., "Who do you think is right? or Do you think this was moral according to this or that law")
   - Predictive questions (e.g., "What do you think could happen next this or that ? ")
3. Keep it interactive like the user feels immersed in the case
4. Do not wait for user answers; proceed directly to the next part of the case after each question.
5. Conclude the output with a summary of the case, its key points, and the actual verdict delivered by the court.

Ensure a conversational tone that engages the user and guides them through the case while prompting thought and reflection on the legal, ethical, and practical dimensions of the case.""")
display(Markdown(f"**Answer:** {output.data}"))

**Answer:** What an intriguing case! Let's dive into the proceedings of **Ramana Dayaram Shetty vs The International Airport Authority of India** decided on May 4, 1979.

**Case Background**
The plaintiff, Ramana Dayaram Shetty, was a passenger who had been forcibly removed from a domestic flight by airport security personnel at Mumbai Airport. Shetty alleged that he was subjected to physical violence and harassment during his removal.

**Step 1: Petition Filed**
Shetty filed a petition under Article 226 of the Indian Constitution, seeking a writ of mandamus to be issued against the International Airport Authority of India (IAAI) and its officials for their actions. He claimed that his fundamental rights were violated, including Articles 14 (equality before law), 19(1)(f) (right to freedom of trade or business), and 21 (protection of life and liberty).

Now, let's analyze the situation:

**Question:** Do you think the IAAI officials overstepped their authority by removing Shetty from the flight?

Please note that we'll proceed with the case without waiting for your response.

**Step 2: Court Proceedings**
The Bombay High Court (now known as the Maharashtra and Goa Common High Court) accepted the petition and issued a notice to the IAAI. The court, comprising Justice V.D. Bhutani and Justice A.N. Grover, heard arguments from both sides.

During the proceedings, it was revealed that Shetty had been removed from the flight allegedly for creating a disturbance and refusing to pay excess baggage charges. However, Shetty denied these allegations and claimed he was innocent.

**Question:** Who do you think is telling the truth in this situation?

In [29]:
!pip install flask flask-cors
!pip install flask flask-cors pyngrok

In [30]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok


In [31]:
!pip install pyngrok
from pyngrok import ngrok


In [32]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok

app = Flask(__name__)  # Changed _name to _name_
CORS(app)  # Enable Cross-Origin Resource Sharing

In [33]:
@app.route('/generate', methods=['POST'])
def generate_response():
    # Get the user input from the POST request
    data = request.json
    Prompt = data.get('input', '')

    # Call your model and get the output
    output = qa(Prompt)

    # Extract the generated text from the model's output
    generated_text = output.data

    # Return the response as JSON
    return jsonify({'output': generated_text})

In [34]:
# Run the Flask server
from threading import Thread

def run_flask():
    app.run(host='0.0.0.0', port=5000)

thread = Thread(target=run_flask)
thread.start()


 * Serving Flask app '__main__'


In [38]:
!ngrok config add-authtoken 2pkmWTbjpz61a8IaCB7GBRYVWEk_4DhaabYvJWpWH9JMZxQK4

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [39]:
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")


Public URL: NgrokTunnel: "https://fe2d-34-125-186-130.ngrok-free.app" -> "http://localhost:5000"
